## Set up directory

In [1]:
%cd /Users/ianleefmans/Desktop/Insight/Project/Re-Identifying_Persistent_Skin_Conditions/skinConditionDetect


/Users/ianleefmans/Desktop/Insight/Project/Re-Identifying_Persistent_Skin_Conditions/skinConditionDetect


# Example using custom datahelper and Faster-RCNN

In [2]:
# import packages
from datahelper import CreateDataset
import torch
from torch.utils.data import DataLoader
from PIL import Image
import os
import torchvision
from model import MVP
from tqdm import tqdm
import os
import pandas as pd
import pickle

TypeError: function() argument 1 must be code, not str

In [ ]:
# Create paths
pickle_path = "/Users/ianleefmans/Desktop/Insight/Project/Re-Identifying_Persistent_Skin_Conditions/skinConditionDetect/annotation_dict.pkl"
data_directory = "/Users/ianleefmans/Desktop/Insight/Project/Data"

# Create dataset
dataset = CreateDataset(pickle_path, data_directory, transform = torchvision.transforms.ToTensor())
def my_collate(batch):
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    #target = torch.LongTensor(target)
    return data, target
train_loader = DataLoader(dataset=dataset, batch_size=4, num_workers=4, shuffle=True, collate_fn=my_collate)
sample = iter(train_loader).next()


In [ ]:
lis =[]
for i in tqdm(range(len(dataset))):
    try:
        dataset[i]
    except:
        lis.append(i)
        print(i)
    finally:
        pass
lis

  0%|          | 46/232811 [00:06<7:47:13,  8.30it/s] 

In [ ]:
image = sample[0]
annotation = sample[1]

In [3]:
# Instantiate Minimum Viable Product (Faster-RCNN)
mvp = MVP()
# Train on example image and annotation
output = mvp(image, annotation)

NameError: name 'MVP' is not defined

In [21]:
output

{'loss_classifier': tensor(1.9176, grad_fn=<NllLossBackward>),
 'loss_box_reg': tensor(0.0086, grad_fn=<DivBackward0>),
 'loss_objectness': tensor(9.6734, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(10.4392, grad_fn=<DivBackward0>)}

In [36]:
import pandas as pd
data_path = "/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/"

test_json = 'user_000000000277.json'

df = pd.read_json(data_path+test_json)



In [ ]:
mvp.eval()
mvp


In [95]:
from tqdm import tqdm
import os
import pandas as pd
import pickle
annotation_dict = {}
count = 0 
for filename in tqdm(os.listdir("/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/")):
    try:
        df = pd.read_json("/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/"+filename)
        df = df.loc[(df.image_details.notnull()) & (df.image_path.notnull()),:]
        for i in list(df.index.values):
            try:
                df.iloc[i].image_details['annotation'] !=None
                df.iloc[i]['image_details']['annotation'][0]
                ok = False
                for j in df.iloc[i].image_details['annotation']:
                    if (j['condition']=='Detected') and ('bounding_boxes' in j.keys()):
                        if len(j['bounding_boxes'])>0:
                            ok = True
                if ok ==False:
                    a = 2/0
                    
                annotation_dict[count] = (filename, i)
                count+=1
            except:
                print
            finally:
                pass
            
    except:
        pass
    finally:
        pass
with open("annotation_dict.pkl", 'wb') as handle:
    pickle.dump(annotation_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 17293/17293 [08:10<00:00, 35.29it/s]


In [15]:
# open pickl3
with open("annotation_dict.pkl", 'rb') as handle:
    dictionary = pickle.load(handle)


In [12]:
#remove bad jsons
bad_json = [2321, 3049, 5296, 6021, 7757, 7797, 8278, 8279, 8757, 9153, 9191]
for i in bad_json:
    dictionary.pop(i)

In [14]:
# save pickle
with open("annotation_dict.pkl", 'wb') as handle:
    pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
len(dictionary) # dictionary length after removing 11 bad json

232800

In [79]:
len(dictionary) # dictionary length before removing 11 bad json


232811